In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 1: Imports + Load CSV into DataFrame
# ─────────────────────────────────────────────────────────────────────────────

# Set up JupyterDash
from jupyter_dash import JupyterDash

# Dash / Plotly imports
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Base64 (for embedding images), pandas, numpy, matplotlib
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For file paths (if needed)
import os

# Load local CSV into a DataFrame (we’re not pulling data from MongoDB in the frontend)
df = pd.read_csv('aac_shelter_outcomes.csv')

# Drop '_id' if present, so it matches our AnimalShelter documents
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


# ─────────────────────────────────────────────────────────────────────────────
# 2: Create JupyterDash App and Hook Up SocketIO
# ─────────────────────────────────────────────────────────────────────────────

# 1) Instantiate the JupyterDash application
app = JupyterDash(__name__)

# 2) Bind Flask-SocketIO to the Dash server
from flask_socketio import SocketIO
socketio = SocketIO(app.server)


# ─────────────────────────────────────────────────────────────────────────────
# 3: Define Dashboard Layout / View (including "Subscribe to Comments")
# ─────────────────────────────────────────────────────────────────────────────

app.layout = html.Div([
    # ---------- App Title & Logo ----------
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(src='data:image/png;base64,{}'.format(
            base64.b64encode(open('Grazioso Salvare Logo.png', 'rb').read()).decode()
        )),
        style={'textAlign': 'center'}
    ),
    html.Center(html.B(html.P('Unique Identifier: JJH340'))),
    html.Hr(),

    # ---------- Animal Type Filter ----------
    html.Label('Animal Type:'),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Dog', 'value': 'Dog'},
            {'label': 'Cat', 'value': 'Cat'}
        ],
        value='All',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),

    # ---------- DataTable of Animal Records ----------
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": col, "id": col, "deletable": False, "selectable": True}
                 for col in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_cell={
            'textAlign': 'left',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 0
        },
        style_cell_conditional=[
            {'if': {'column_id': 'name'}, 'width': '20%'},
            {'if': {'column_id': 'breed'}, 'width': '30%'}
        ],
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
        ],
        style_header={'backgroundColor': 'rgb(230, 230, 230)',
                      'fontWeight': 'bold'}
    ),

    html.Br(),

    # ─────────── NEW SECTION: Subscribe to Live Comments ───────────
    html.Label("Subscribe to Live Comments:"),
    dcc.Checklist(
        id='subscribe-comments',
        options=[{'label': 'Yes, show me live comments', 'value': 'ON'}],
        value=[],
        labelStyle={'display': 'inline-block', 'margin-right': '10px'}
    ),
    html.Div(
        id='comments-panel',
        style={'border': '1px solid #ccc', 'padding': '10px', 'margin': '10px 0'}
    ),
    html.Br(),
    # ────────────────────────────────────────────────────────────────

    html.Hr(),

    # ---------- Graph + Map Layout ----------
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                html.Div(id='graph-id', className='col s12 m6'),
                html.Div(id='map-id', className='col s12 m6')
             ]),

    # A hidden dummy output to hook both clientside callbacks
    html.Div(id='dummy-output', style={'display':'none'})
])


# ─────────────────────────────────────────────────────────────────────────────
# 4: Python Callback for Filtering the DataTable by Animal Type
# ─────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    """
    If the user selects "Dog" or "Cat", filter the DataFrame accordingly.
    Otherwise show all records.
    """
    if filter_type == 'All':
        return df.to_dict('records')
    return df[df['animal_type'] == filter_type].to_dict('records')


# ─────────────────────────────────────────────────────────────────────────────
# 5: Python Callback for Posting a Comment (Server-Side)
# ─────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('new-comment', 'value'),
    Input('btn-post-comment', 'n_clicks'),
    State('new-comment', 'value')
)
def post_comment(n_clicks, comment_text):
    """
    When the user clicks "Post Comment," call AnimalShelter.create_comment(...)
    so that the server stores it in MongoDB and broadcasts 'new_comment'.
    Then clear the input box.
    """
    if n_clicks and comment_text:
        # 1) Instantiate AnimalShelter with the Flask app so SocketIO works
        shelter = AnimalShelter(
            username="YOUR_USERNAME",
            password="YOUR_PASSWORD",
            host="localhost",
            port=27017,
            db="YOUR_DB_NAME",
            collection="YOUR_COLLECTION_NAME",
            app=app.server
        )
        # 2) Build the comment object with a timestamp (and any other fields)
        comment_obj = {
            "text": comment_text,
            "timestamp": str(pd.Timestamp.now())
        }
        # 3) Insert & broadcast on 'new_comment'
        shelter.create_comment(comment_obj)

        # 4) Clear the text area
        return ""

    # If no click or no text, leave the box as-is
    return comment_text


# ─────────────────────────────────────────────────────────────────────────────
# 6: Client-Side Callback to Listen for 'new_data' (Unchanged from Module Two)
# ─────────────────────────────────────────────────────────────────────────────

app.clientside_callback(
    """
    function(data) {
        if (window.socket) {
            window.socket.on('new_data', function(record) {
                // Find the Dash DataTable component on the page
                let tbl = window.dash_clientside.callback_context
                          .outputs_list.filter(o => o.id=='datatable-id')[0];
                let current = tbl.props.data;
                // Append the new record to the existing table data
                current.push(record);
                tbl.setProps({data: current});
            });
        }
        return '';
    }
    """,
    Output('dummy-output', 'children'),
    Input('datatable-id', 'data')
)


# ─────────────────────────────────────────────────────────────────────────────
# 7: Client-Side Callback to Listen for 'new_comment' (Opt-In Logic)
# ─────────────────────────────────────────────────────────────────────────────

app.clientside_callback(
    """
    function(subscribeValues) {
        if (!window.socket) {
            console.warn("Socket.IO not initialized.");
            return '';
        }
        // If the user ticks the box (value contains 'ON'), then subscribe
        if (subscribeValues && subscribeValues.indexOf('ON') !== -1) {
            // Attach listener for 'new_comment'
            window.socket.on('new_comment', function(comment) {
                let panel = document.getElementById('comments-panel');
                let p = document.createElement('p');
                // We assume comment has a 'text' property
                p.textContent = comment.text;
                panel.appendChild(p);
            });
        } else {
            // If unchecked, remove the 'new_comment' listener so no more comments arrive
            window.socket.off('new_comment');
        }
        return '';
    }
    """,
    Output('dummy-output','children'),
    Input('subscribe-comments','value')
)


# ─────────────────────────────────────────────────────────────────────────────
# 8: Graph-Updating Callback (Pie Chart of Breed Distribution)
# ─────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    """
    Whenever the table data changes (e.g. new_data broadcast
    or a filter is applied), update the pie chart of breed distribution.
    """
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to generate chart.")]

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]


# ─────────────────────────────────────────────────────────────────────────────
# 9: Style Highlighting Callback (Highlight Selected Column)
# ─────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', "selected_columns")
)
def update_styles(selected_columns):
    """
    Highlight the selected columns in light blue.  If no column
    is selected, return no extra styling.
    """
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]


# ─────────────────────────────────────────────────────────────────────────────
# 10: Map-Updating Callback (Show Map for Selected Row)
# ─────────────────────────────────────────────────────────────────────────────

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    """
    Whenever the user selects a row in the table, show a Leaflet map
    (dash_leaflet) centered on that animal's (lat, lon).  If invalid
    or missing coordinates, show an error message.
    """
    if viewData is None or selected_rows is None or len(viewData) == 0:
        return [html.P("No data available to display on the map.")]

    dff = pd.DataFrame.from_dict(viewData)

    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        idx = selected_rows[0] if selected_rows else 0
        try:
            lat = float(dff.iloc[idx]['location_lat'])
            lon = float(dff.iloc[idx]['location_long'])
            if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
                raise ValueError("Coordinates out of range.")
        except (ValueError, TypeError, KeyError):
            return [html.P("Invalid location data; cannot display on the map.")]

        return [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[lat, lon],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[lat, lon],
                        children=[
                            dl.Tooltip(dff.iloc[idx]['breed']),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[idx]['name'])
                            ])
                        ]
                    )
                ]
            )
        ]
    else:
        return [html.P("No data available to display on the map.")]


# ─────────────────────────────────────────────────────────────────────────────
# 11: Run the Dash App (External Mode on Port 8050)
# ─────────────────────────────────────────────────────────────────────────────

if __name__ == '__main__':
    # Use 'mode="external"' so it pops open in a browser tab
    app.run_server(mode='external', port=8050, debug=True)


/home/jameshatzibir_snhu/.local/lib/python3.9/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
